In [4]:
import pickle
import numpy as np
import multiprocessing
import numpy as np
import tqdm
import os
from pyts.approximation import PAA
from pyts.image import GADF, MTF, RecurrencePlots, GASF
from sklearn.preprocessing import MinMaxScaler
import time
import gc
import os
import csv
import warnings
import shutil
from functools import partial

warnings.simplefilter(action='ignore', category=FutureWarning)

Xd = pickle.load(open("RML2016.10a_dict.pkl",'rb'), encoding='latin1')
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]): lbl.append((mod,snr))
X = np.vstack(X)
# plt.ioff()
path = os.path.join(os.getcwd())
signals = X

dict_R = []
dict_G = []
dict_B = []


In [26]:
def runnerr(j):
    SNR = lbl[j][1]
    array = X[j]
    IMG_SIZE = 128
    encoder1 = GASF(IMG_SIZE)
    encoder2 = MTF(IMG_SIZE, n_bins=IMG_SIZE//10, quantiles='gaussian')
    encoder3 = GADF(IMG_SIZE)
    Amp = np.sqrt(np.add(np.square(X[j][0]), np.square([X[j][1]])))    
    r = np.squeeze(encoder1.fit_transform(array)) 
    g = np.squeeze(encoder2.fit_transform(array)) 
    b = np.squeeze(encoder3.fit_transform(Amp)) 
    # assert(r.shape==b.shape==g.shape)
    scaler = MinMaxScaler(feature_range=(0, 1))
    shape = r.shape
    r = scaler.fit_transform(r.reshape(-1, 1)).reshape(shape)[1, :] #Q
    g = scaler.fit_transform(g.reshape(-1, 1)).reshape(shape)[0, :] #I
    b = scaler.fit_transform(b) #Amplitude
        
    if SNR not in dict_R:
        dict_B[SNR] =[b]
        dict_G[SNR] =[g[0]]
        dict_R[SNR] =[r[1]]
        return
    dict_B[SNR].append(b)
    dict_G[SNR].append(g[0])
    dict_R[SNR].append(r[1])

for i in tqdm.tqdm(range(20000)):
    runnerr(i)


100%|██████████| 20000/20000 [00:52<00:00, 382.81it/s]


In [29]:
print(dict_R.keys())
print(dict_G.keys())
print(dict_B.keys())

dict_keys([-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18])
dict_keys([-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18])
dict_keys([-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18])


In [30]:
f = open("results.csv", "w")
writer = csv.DictWriter(f, fieldnames=['SNRs','Red-Green', 'Blue-Green', 'Red-Blue'])
writer.writeheader()
f.close()

In [32]:
with open('results.csv', 'a+', newline='') as f:
    f = csv.writer(f)
    for n in range(-20, 20, 2):
        lst = [n]
        rg = ((np.mean(dict_R[n])-np.mean(dict_G[n]))**2)/((np.var(dict_R[n]))**2 - (np.var(dict_G[n])**2))
        bg = ((np.mean(dict_B[n])-np.mean(dict_G[n]))**2)/((np.var(dict_B[n]))**2 - (np.var(dict_G[n])**2))
        rb = ((np.mean(dict_R[n])-np.mean(dict_B[n]))**2)/((np.var(dict_R[n]))**2 - (np.var(dict_B[n])**2))
        lst.append(rg)
        lst.append(bg)
        lst.append(rb)
        f.writerow(lst)